# Agent配置
本节目的，学习三个最原始的组件：
`Agents`，它是配备指令和工具的大型语言模型
`Handoffs`，它允许代理将特定任务委派给其他代理
`Guardrails`，它可以对输入到代理的内容进行验证

## 多Agent的目的
解决：
- 专业化分工
- 任务路由与分发：决策Agent->子Agent
- 安全与合规:Agent->Guardrails

## 三个核心组件
1. `Agents`智能体：
- 赋予特定指令`instructions`
- 模型`model`的实例
- 配备工具`tools`
2. `Handoffs`任务交接；
- 一种机制，允许代理将特定任务委派给其他代理
- `handoff_description`：描述了任务交接的详细信息，包括任务的类型、目标和任何相关的上下文。相当于模型的能力简介/名片。
- `Handoffs`：一个列表，写了其下属设有的其他`Agents`的`handoff_description`。
3. `Guardrails`护栏：
- 一种验证和过滤层，处理Agent的输入和输出检查
- 例如：`input_guardrails`可以检查输入是否符合预期的格式或内容，不符合则进行拦截，不传入该Agent；而`output_guardrails`可以检查输出是否符合预期的格式或内容。

## 设计与实现（Design&Implementation）：构建一个多Agent系统

### Step1:确定LLM（选择“大脑）
如何使用`LitellmModel`来灵活配置Mistral或DeepSeek模型。这是为我们所有的Agent提供“思考能力”的基础。

In [ ]:
# 代码作用：配置一个Mistral模型作为所有Agent的“大脑”
from agents.extensions.models.litellm_model import LitellmModel
import os
from dotenv import load_dotenv
# 加载环境变量
load_dotenv()
# 从环境变量中获取Mistral API密钥
chat_model = "mistral-small-latest"
base_url = "https://api.mistral.ai/v1"
api_key = os.getenv('mistral_key')

llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

In [ ]:
# 使用DeepSeek模型
chat_model = "deepseek/deepseek-chat"
base_url = "https://api.deepseek.com"
api_key = os.getenv('DEEPSEEK_API_KEY')

### Step2: 构建多Agent系统
配置两个不同技能的Agent

In [ ]:
from agents import Agent

# 历史老师Agent
history_tutor_agent = Agent(
    name="History Tutor",
    # “名片”：告诉别人我擅长历史问题
    handoff_description="Specialist agent for historical questions",
    # “工作手册”：详细指令
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
    model=llm,
)

# 数学老师Agent
math_tutor_agent = Agent(
    name="Math Tutor",
    # “名片”：告诉别人我擅长数学问题
    handoff_description="Specialist agent for math questions",
    # “工作手册”：详细指令
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
    model=llm,
)

### Step3: 构建“护栏”Agent
定义安全规则

In [ ]:
from agents import GuardrailFunctionOutput, InputGuardrail, Agent, Runner
from pydantic import BaseModel

# 定义一个数据结构，用于规范“护栏”的输出
class HomeworkOutput(BaseModel): # 一般写为GuardrailOutput
    is_homework: bool  # 这个问题是不是作业？
    reasoning: str     # 判断的理由是什么？

# “保安”Agent，它的唯一任务就是判断问题类型
guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking about homework.",
    output_type=HomeworkOutput, # 强制它的输出符合HomeworkOutput格式
    model=llm,
)

# 定义护栏的具体行为：一个异步函数
async def homework_guardrail(ctx, agent, input_data):
    # 1. 让“保安”Agent去判断输入的问题
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context) # context 是为了让“保安”Agent有上下文信息
    # 2. 将结果解析为我们定义的HomeworkOutput格式
    # 目的：LLM返回的虽然按照HomeworkOutput格式，但是它是一个字符串，我们需要将其转换为结构化的Python对象，而该函数的作用就是实现这一转换。
    # - 解析：接受模型返回的原始字符串
    # - 验证：使用HomeworkOutput作为模版，去验证解析结果符合预期格式
    # - 转换：如果验证通过，将字符串转换为结构化的Python对象
    final_output = result.final_output_as(HomeworkOutput)
    # 3. 返回一个标准格式的输出
    return GuardrailFunctionOutput(
        output_info=final_output,
        # 关键：如果 is_homework 为 False，就触发“警报”(tripwire_triggered=True)
        tripwire_triggered=not final_output.is_homework,
    )
        # 此处的tripwire_triggered是做了一个bool计算，如果is_homework为False，那么tripwire_triggered就为True，则输出结果。

### Step4: 构建“作业”Agent并且安装“护栏”
创建的Agent相当于项目经理，告诉其可以管理哪些”Agent“，并且安装上“护栏”

In [ ]:
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    # “下属列表”：告诉它手下有历史和数学两位专家
    handoffs=[history_tutor_agent, math_tutor_agent],
    # “安保系统”：在处理任何输入前，先经过homework_guardrail的检查
    input_guardrails=[
        InputGuardrail(guardrail_function=homework_guardrail),
    ],
    model=llm,
)

### 完整的执行流程
当你运行 `await Runner.run`(triage_agent, "一个问题") 时，会发生以下事情：
1. 护栏检查：`triage_agent`首先触发`homework_guardrail`。
2. 保安出动：`guardrail_agent`被调用，判断“一个问题”是不是作业相关。
3. 触发警报？
- 是作业 (`is_homework: true`)：`tripwire_triggered`为`False`，护栏通过，流程继续。
- 不是作业 (`is_homework: false`)：`tripwire_triggered`为`True`，护栏拦截，程序抛出`InputGuardrailTripwireTriggered`异常，流程终止。
4. 经理决策：如果护栏通过，`triage_agent`会分析问题，并根据`history_tutor_agent`和`math_tutor_agent`的“名片”（`handoff_description`）来决定把任务**交接（Handoff）**给谁。
5. 专家执行：被选中的专家Agent（比如`history_tutor_agent`）接收任务并生成最终答案。
6. 返回结果：专家Agent的答案作为最终结果返回。

## 代码优化与实践
Jupyter运行：
- 去掉`async def main()`和`if __name__ == "__main__"`部分
- 直接在单元格顶层使用`await Runner.run(...)`即可。
异常捕获：
- 为了处理可能的异常，建议在单元格中使用`try-except`块来捕获`InputGuardrailTripwireTriggered`异常。

In [ ]:
# Jupyter 运行
result = await Runner.run(triage_agent, "does drinking tee good for the body?")
print(result.final_output)

In [ ]:
# 异常捕获
from agents import Runner
from agents.exceptions import InputGuardrailTripwireTriggered
try:
    result = await Runner.run(triage_agent, "如何证明勾股定理？")
    print("结果:", result.final_output)
except InputGuardrailTripwireTriggered as e:
    print("护栏触发：你的问题不是作业相关")

### 测试
提不同的问题，让它回答，看看输出什么结果：
```
如何证明勾股定理？
does drinking tee good for the body?
who was the first president of the united states?
what is life?
```
实际使用，该Agent很容易拦截了一些通用问题，并且定性为非Homework。
个人修改了homework_guardrail函数，添加了调试日志，用于查看Guardrail的判断结果。

In [ ]:
async def homework_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeworkOutput)
    # 调试日志
    print(f"[Guardrail] is_homework={final_output.is_homework} | reasoning={final_output.reasoning}")
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_homework,
    )